In [1]:
import redis
import faker as fk
import pandas
import random
fake = fk.Faker()

# main client
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

In [2]:
def make_rand_computer():
    os = ['windows', 'macOS', 'linux']
    return {"ip": fake.ipv4(), "name": fake.user_name(), "os": random.choice(os)}

def make_rand_mac():
    return fake.mac_address()

def unravel_dict(dictionary):
    return " ".join([f"{key} {value}"for key,value in dictionary.items()])

def add_computer(computer):
    mac = make_rand_mac()
    r.execute_command(f"HSET mac:{mac} {unravel_dict(computer)}")
    r.execute_command(f"HSET index.mac.by.ip {computer['ip']} {mac}")
   
    macs = r.execute_command("HGETALL index.mac.by.ip")
    macs = macs[1::2]
    macs.remove(mac)
    for _ in range(4):
        if macs == []:
            break
        connected = random.choice(macs)
        r.execute_command(f"SADD connections:{mac} {connected}")
        r.execute_command(f"SADD connections:{connected} {mac}")

def initialize_content(n):
    for _ in range(n):
        add_computer(make_rand_computer())


In [ ]:
def show_all_macs():
    for key in r.scan_iter("mac:*"):
        print(key)
    
def show_all_computers():
    for key in r.scan_iter("mac:*"):
        print(key)
        print(r.execute_command(f"HGETALL {key}"))
        
def show_all_connections():
    for key in r.scan_iter("mac:*"):
        mac = key[len("mac:"):]
        print(mac)
        print(r.execute_command(f"SMEMBERS connections:{mac}"))
        
def change_ip(old_ip, new_ip):
    r.execute_command(f"WATCH index.mac.by.ip")
    mac_address = r.execute_command(f"HGET index.mac.by.ip {old_ip}")
    r.execute_command(f"MULTI")
    r.execute_command(f"HSET index.mac.by.ip {new_ip} {mac_address}")
    r.execute_command(f"HDEL index.mac.by.ip {old_ip}")
    r.execute_command(f"HSET mac:{mac_address} ip {new_ip}")
    r.execute_command('PUBLISH', 'computer.logs', f"IP of computer {mac_address} has changed from {old_ip} to {new_ip}")
    r.execute_command(f"EXEC")

In [9]:
initialize_content(10)

In [12]:
show_all_computers()

mac:88:19:9e:a3:60:bf
['ip', '119.22.49.00', 'name', 'dalereed', 'os', 'windows']
mac:fc:6a:57:b0:50:0b
['ip', '107.0.21.181', 'name', 'peterwalter', 'os', 'macOS']
mac:ac:ab:82:9d:92:86
['ip', '93.34.102.150', 'name', 'jeffrey80', 'os', 'windows']
mac:e8:83:0a:3b:ad:79
['ip', '171.198.171.109', 'name', 'buckmelinda', 'os', 'windows']
mac:02:d6:de:0b:28:35
['ip', '217.153.98.160', 'name', 'ilopez', 'os', 'linux']
mac:32:ca:90:cd:ae:14
['ip', '171.91.237.30', 'name', 'odickson', 'os', 'macOS']
mac:9c:14:09:d8:94:64
['ip', '88.53.130.15', 'name', 'janice64', 'os', 'windows']
mac:2c:9a:6c:a3:b7:62
['ip', '26.15.217.138', 'name', 'wilsonjohn', 'os', 'linux']
mac:14:95:6c:58:d5:4d
['ip', '194.134.86.131', 'name', 'dnichols', 'os', 'macOS']
mac:e6:97:e8:94:1e:9f
['ip', '44.34.140.187', 'name', 'averyeric', 'os', 'windows']


In [11]:
change_ip("119.22.49.29","119.22.49.00")

In [6]:
r.flushdb()

True